# estimated ramge SOH
The goal of this notebook is to compute the soh estimated_range, charge_energy_added and soc.

## Setup

### Imports

In [ ]:
import scipy.interpolate as interpolate
import scipy.optimize as optimize
import pandas as pd
from pandas import DataFrame as DF
from pandas import Series
import plotly.express as px
import numpy as np

from analysis.tesla.tesla_fleet_info import get_fleet_info
from analysis.tesla.tesla_raw_tss import get_raw_tss
from analysis.tesla.tesla_constants import *
from core.time_series_processing import compute_cum_integrals_of_current_vars, perf_mask_and_idx_from_condition_mask
from core.pandas_utils import floor_to, uniques_as_series, series_start_end_diff
from core.plt_utils import plt_3d_df

### Data extraction

In [ ]:
fleet_info = get_fleet_info()
raw_tss = get_raw_tss()
raw_tss.loc[:, ["model", "default_capacity"]] = fleet_info.loc[raw_tss["vin"], ["model", "default_kwh_energy_capacity"]].values # Use .values so that pandas ignores the index

### Preprocessing

In [ ]:
def process_ts(raw_ts:DF) -> DF:
    return (
        raw_ts
        .assign(
            ffilled_odometer=raw_ts["odometer"].ffill(),
            floored_soc=floor_to(raw_ts["soc"].ffill(), 1),
            date_diff=raw_ts["date"].diff(),
            soc_diff=raw_ts["soc"].diff(),
        )
        .sort_values(by="date")
        .pipe(compute_cum_integrals_of_current_vars)
        .pipe(perf_mask_and_idx_from_condition_mask, "in_charge")
        .pipe(perf_mask_and_idx_from_condition_mask, "in_dishcarge")
        .assign(energy_diff=lambda df: df["cum_energy"].diff())
    )

tss:DF = (
    raw_tss
    .rename(columns={
        "battery_level": "soc",
    })
    .assign(
        in_charge=raw_tss["charging_state"].eq("Charging"),
        in_dishcarge=raw_tss["charging_state"].eq("Disconnected"),
        ffiled_outside_temp=raw_tss["outside_temp"].ffill(),
        ffiled_inside_temp=raw_tss["inside_temp"].ffill(),
    )
    .groupby("vin")
    .apply(process_ts, include_groups=False)
)

Let's check the sparcity of the data we will need to estimate the SOH:

In [ ]:
tss[["charge_miles_added_ideal", "charge_energy_added", "battery_range", "soc"]].count() / len(tss)

Great, we won't need to do any further preprocessing.

## SOH estimation

In [ ]:
tss:DF = tss.eval("soh = ((charge_energy_added / charge_miles_added_ideal) * battery_range) / soc")
tss["vin"] = tss.index.get_level_values(0)

In [ ]:
fig = px.scatter(
    tss[~np.isinf(tss["soh"])].query("inside_temp < 45"),
    x="odometer",
    y="soh",
    color="vin",
    color_continuous_scale="Rainbow",
    trendline="ols",
    opacity=0.3,
)
fig.update_traces(line={"color": "black", "dash": "dash"})
fig.update_layout(height=1000)